<a href="https://colab.research.google.com/github/behigu/Challenge_-Day2/blob/main/Task_3_Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyLDAvis

In [20]:
pip install numpy

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint
import string
import os
import re

# visual
import matplotlib.pyplot as plt
import seaborn as sns

# filter out noise words and more clean up on word
from wordcloud import STOPWORDS,WordCloud
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# gensim
import gensim
from gensim.models import CoherenceModel
from gensim import corpora

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/behigu/Challenge_-Day2/main/output.csv')
df.head()

,Unnamed: 0,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548,612,ketuesriche,551,351,None,NaN,TelGlobalHealth\nWHOAFRO,Mass
1,1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195,92,Grid1949,66,92,None,NaN,globalhlthtwit,"Edinburgh, Scotland"
2,2,Fri Jun 18 17:56:07 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Thank you @research2note for creating this ama...,0.316667,0.483333,en,2,1,LeeTomlinson8,1195,1176,None,red4research,NHSRDForum\nResearch2note\nNHSRDForum,NaN
3,3,Fri Jun 18 17:56:10 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580,899,RIPNY08,2666,2704,None,NaN,HighWireTalk,NaN
4,4,Fri Jun 18 17:56:20 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72,20,pash22,28250,30819,None,NaN,PeterHotez,United Kingdom


In [8]:
print(df.shape)
print(df.columns)

(6532, 16)
Index(['Unnamed: 0', 'created_at', 'source', 'original_text', 'polarity',
       'subjectivity', 'lang', 'favorite_count', 'retweet_count',
       'original_author', 'followers_count', 'friends_count',
       'possibly_sensitive', 'hashtags', 'user_mentions', 'place'],
      dtype='object')


In [9]:
# check if we have null 
count_texts = df['original_text'].isnull().sum()
print("Data Frame before removing null rows ", df.shape)
print("Number of null orignal_text ", count_texts)

Data Frame before removing null rows  (6532, 16)
Number of null orignal_text  0


In [10]:
# clean out unwanted values from the text, like links and other staff
import re
def clean_text(text):
    hash_tag_removed = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    removed_links = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', hash_tag_removed, flags=re.MULTILINE)
    result = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', removed_links)
    return result

df['original_text'] = df['original_text'].apply(clean_text)

<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<ipython-input-10-cc8200d889d7>:5: DeprecationWarning: invalid escape sequence \(
  removed_links = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', hash_tag_removed, flags=re.MULTILINE)


In [11]:
# SETUP NEW DATA FRAME FOR CLEANED DATA
cleanTweet = pd.DataFrame()
cleanTweet['clean_text'] = df['original_text']
cleanTweet['polarity'] = df['polarity']

cleanTweet.head()

,clean_text,polarity
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333
2,Thank you for creating this amazing campaign ...,0.316667
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111
4,I think it’s important that we don’t sell COVA...,0.280000


In [12]:
cleanTweet.columns

Index(['clean_text', 'polarity'], dtype='object')

In [13]:
cleanTweet.dropna()

,clean_text,polarity
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333
2,Thank you for creating this amazing campaign ...,0.316667
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111
4,I think it’s important that we don’t sell COVA...,0.280000
...,...,...
6527,Zweli please just release the graphic designer...,0.000000
6528,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111
6529,"Respectfully, veterinarians have to have a b...",0.281250
6530,"""Africa needs millions more doses here &amp; n...",0.125000


In [14]:
# Noise words need to be removed.
def process_data():
    cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: x.lower())
    cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    sentence_list = [tweet for tweet in cleanTweet['clean_text']]
    word_list = [sentence.split() for sentence in sentence_list]
    
    word_to_id = corpora.Dictionary(word_list)
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return word_list, word_to_id, corpus_1

In [15]:
word_list, id2word, corpus = process_data()
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 3), (21, 1), (22, 2), (23, 1), (24, 4), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [18]:
#### pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.057708 -0.097534       1        1  31.233046
0     -0.162856 -0.138606       2        1  18.764370
1     -0.088946  0.332325       3        1  18.734848
2     -0.039894 -0.089817       4        1  17.994297
3      0.349403 -0.006369       5        1  13.273439, topic_info=      Term         Freq        Total Category  logprob  loglift
54      to  5863.000000  5863.000000  Default  30.0000  30.0000
273  india  1167.000000  1167.000000  Default  29.0000  29.0000
86     you  1923.000000  1923.000000  Default  28.0000  28.0000
162   your  1133.000000  1133.000000  Default  27.0000  27.0000
6       by  1530.000000  1530.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
147    are   240.709717  1858.522290   Topic5  -4.5604  -0.0245
24     the   237.333180  6173.958129   Topic5  -4.5745  -1.2392
175